In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import ast
import matplotlib.pyplot as plt
import random
from datetime import datetime as dt
from random import randint
from collections import Counter
import math
import sklearn as sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import *
from sklearn.metrics.pairwise import linear_kernel
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import roc_curve, auc
from random import randint
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
users = pd.read_csv('users.csv', encoding = "ISO-8859-1")
articles = pd.read_csv('articles.csv', encoding = "ISO-8859-1")

In [ ]:
# take interactions that have sharerHashes
forgraph_t = articles[articles['sharerHash'] != '[]']
forgraph_t = forgraph_t.reset_index()

In [ ]:
# take interactions that have userHashes that are in the users table
users_list = set(users["userHash"])
forgraph = forgraph_t[forgraph_t["userHash"].isin(users_list)] 

In [ ]:
def getRandomInteraction(all_interactions):
    index = randint(0, len(all_interactions)-1)
    currUserHash = all_interactions.iloc[[index]]['userHash'][index]
    currSharerHash_list = ast.literal_eval(all_interactions.iloc[[index]]['sharerHash'][index])
    currSeenTime_list = ast.literal_eval(all_interactions.iloc[[index]]['articleSeenTime'][index])
    if len(currSharerHash_list) == 1:
        currSharerHash = currSharerHash_list[0]
        currSeenTime = currSeenTime_list[0]
    else:
        t_index = randint(0, len(currSharerHash_list)-1)
        currSharerHash = currSharerHash_list[t_index]
        currSeenTime= currSeenTime_list[t_index]
    return [str(currUserHash), str(currSharerHash), str(currSeenTime)]

In [ ]:
def thematicInterest(rand_userHash, users, rand_user_article):
    # get article data
    article_data = getArticleData(rand_user_article)
    #article_counts = Counter(stem(removeStopwords(tokenize(article_data))))
    #print (article_data)
    
    u_data = getUserData(rand_userHash, users)
    #user_counts = Counter(stem(removeStopwords(tokenize(u_data))))
    #print (u_data)
    
    docs = [article_data, u_data]
    tfidf = TfidfVectorizer(stop_words='english')
    output = tfidf.fit_transform(docs)
    cosine_similarities = linear_kernel(output[0:1], output).flatten()
    tfidf = TfidfVectorizer(stop_words='english')
    output = tfidf.fit_transform(docs)
    cosine_similarities = linear_kernel(output[0:1], output).flatten()
    return cosine_similarities[1]

In [ ]:
def trust(x, y, article_data):
    x_seen = article_data[article_data["userHash"] == int(x)]
    x_seen_from_y = pd.DataFrame()
    for i in range(0, len(x_seen)):
        temp_sharer_list = ast.literal_eval(x_seen.iloc[i]["sharerHash"])
        for s in temp_sharer_list:
            if int(s) == int(y):
                x_seen_from_y.append(x_seen.iloc[i])
                break
    if (len(x_seen_from_y) == 0):
        return 0
    else:
        return len(x_seen_from_y[x_seen_from_y["articleClicked"] == True])/len(x_seen_from_y)

In [ ]:
def engagementHistory(x, rand_article, article_data):
    x_seen = article_data[article_data["userHash"] == int(x)]
    x_engaged = x_seen[x_seen["articleClicked"] == True].reset_index()

    timestamps = []
    for index, row in valid_interactions.iterrows():
        curr_time = ast.literal_eval(row["articleSeenTime"])[len(ast.literal_eval(row["articleSeenTime"]))-1]
        timestamps.append(dt.strptime(curr_time, "%a, %d %b %Y %H:%M:%S %Z"))

    timestamps_df = pd.DataFrame({'timestamps': timestamps})
    x_engaged_t = x_engaged.join(timestamps_df)
    x_engaged = x_engaged_t.sort_values(["timestamps"], ascending=False)
    timedelta = getilogtimedelta(x, rand_article, article_data)
    past_engagement_data = ""
    for i in range(0, timedelta):
        past_engagement_data = past_engagement_data + curr_article_data

In [ ]:
rand_article_data = getArticleData(rand_article)

    docs = [rand_article_data, past_engagement_data]
    tfidf = TfidfVectorizer(stop_words='english')
    output = tfidf.fit_transform(docs)
    cosine_similarities = linear_kernel(output[0:1], output).flatten()
    tfidf = TfidfVectorizer(stop_words='english')
    output = tfidf.fit_transform(docs)

    cosine_similarities = linear_kernel(output[0:1], output).flatten()
    return cosine_similarities[1]

In [ ]:
def getArticleData(rand_article):
    comments = ast.literal_eval(rand_article["articleComments"])[0]
    title = rand_article["articleTitle"]
    if (comments == "NULL"):
        comments = ""
    if (title == "NULL"):
        title = ""
    article_data = str(comments) + " " + str(title)
    return article_data

def getUserData(userHash, users): #user_info is a df that is a single row
    user_info = users[users["userHash"] == userHash].reset_index()
    u_data = ""
    if str(user_info["currentLocation"][0]) != "nan":
        u_data = u_data + str(user_info["currentLocation"][0]) + " "
    if str(user_info["education1location"][0]) != "nan":
        u_data = u_data + str(user_info["education1location"][0]) + " "
    if str(user_info["education1name"][0]) != "nan":
        u_data = u_data + str(user_info["education1name"][0]) + " "
    if str(user_info["education2location"][0]) != "NaN":
        u_data = u_data + str(user_info["education2location"][0]) + " "
    if str(user_info["education2name"][0]) != "NaN":
        u_data = u_data + str(user_info["education2name"][0]) + " "
    if str(user_info["education3location"][0]) != "NaN":
        u_data = u_data + str(user_info["education3location"][0]) + " "
    if str(user_info["education3name"][0]) != "NaN":
        u_data = u_data + str(user_info["education3name"][0]) + " "
    if str(user_info["gender"][0]) != "NaN":
        u_data = u_data + str(user_info["gender"][0]) + " "
    if str(user_info["hometown"][0]) != "NaN":
        u_data = u_data + str(user_info["hometown"][0]) + " "
    if str(user_info["language1"][0]) != "nan":
        u_data = u_data + str(user_info["language1"][0]) + " "
    if str(user_info["language2"][0]) != "nan":
        u_data = u_data + str(user_info["language2"][0]) + " "
    if str(user_info["language3"][0]) != "nan":
        u_data = u_data + str(user_info["language3"][0]) + " "
    if str(user_info["otherLocation1"][0]) != "nan":
        u_data = u_data + str(user_info["otherLocation1"][0]) + " "
    if str(user_info["otherLocation2"][0]) != "nan":
        u_data = u_data + str(user_info["otherLocation2"][0]) + " "
    if str(user_info["work1location"][0]) != "nan":
        u_data = u_data + str(user_info["work1location"][0]) + " "
    if str(user_info["work1name"][0]) != "nan":
        u_data = u_data + str(user_info["work1name"][0]) + " "
    if str(user_info["work1role"][0]) != "nan":
        u_data = u_data + str(user_info["work1role"][0]) + " "
    if str(user_info["work2location"][0]) != "nan":
        u_data = u_data + str(user_info["work2location"][0]) + " "
    if str(user_info["work2name"][0]) != "nan":
        u_data = u_data + str(user_info["work2name"][0]) + " "
    if str(user_info["work2role"][0]) != "nan":
        u_data = u_data + str(user_info["work2role"][0]) + " "
    if str(user_info["work3location"][0]) != "nan":
        u_data = u_data + str(user_info["work3location"][0]) + " "
    if str(user_info["work3name"][0]) != "nan":
        u_data = u_data + str(user_info["work3name"][0]) + " "
    if str(user_info["work3role"][0]) != "nan":
        u_data = u_data + str(user_info["work3role"][0]) + " "
    if str(user_info["work4location"][0]) != "nan":
        u_data = u_data + str(user_info["work4location"][0]) + " "
    if str(user_info["work4name"][0]) != "nan":
        u_data = u_data + str(user_info["work4name"][0]) + " "
    if str(user_info["work4role"][0]) != "nan":
        u_data = u_data + str(user_info["work4role"][0]) + " "
    return u_data

def getUserArticleSim(article_data, u_data):
    docs = [article_data, u_data]
    tfidf = TfidfVectorizer(stop_words='english')
    output = tfidf.fit_transform(docs)
    cosine_similarities = linear_kernel(output[0:1], output).flatten()
    return cosine_similarities

In [ ]:
features = []

In [ ]:
users_userHashes = users["userHash"]
articles_userHashes = forgraph["userHash"]
valid_userHashes = list(set(users_userHashes) & set(articles_userHashes))

valid_interactions = forgraph[forgraph["userHash"].isin(valid_userHashes)]

In [ ]:
count = 0
while len(features) < 1000:
    print(count)
    random_userHash = random.choice(valid_userHashes)
    user_articles = valid_interactions[valid_interactions['userHash'] == random_userHash]
    engaged_articles = user_articles[user_articles["articleClicked"] == True]
    random_valid_article = engaged_articles.sample(n=1).reset_index().iloc(0)
    random_valid_article = random_valid_article[0]
    
    currSharerHash = ast.literal_eval(random_valid_article["sharerHash"])[0]
    
    timestamp = ast.literal_eval(random_valid_article["articleSeenTime"])[0]
    timestamp = dt.strptime(timestamp, "%a, %d %b %Y %H:%M:%S %Z")
    beforetime_DF = pd.DataFrame()
    for index, row in valid_interactions.iterrows():
        for i in ast.literal_eval(row["articleSeenTime"]):
            date_obj = dt.strptime(i, "%a, %d %b %Y %H:%M:%S %Z")
            if date_obj <= timestamp:
                beforetime_DF = beforetime_DF.append(row, ignore_index=True)
                break
    
    not_engaged_articles = beforetime_DF[beforetime_DF["articleClicked"] == False]
    random_notengaged_article = not_engaged_articles.sample(n=1).reset_index().iloc(0)
    random_notengaged_article = random_notengaged_article[0]
    notengaged_sharerHash = ast.literal_eval(random_notengaged_article["sharerHash"])[0]
    
    s_t = thematicInterest(random_userHash, users, random_valid_article)
    s_f = thematicInterest(random_userHash, users, random_notengaged_article)
    t_t = trust(random_userHash, currSharerHash, beforetime_DF)
    t_f = trust(random_userHash, notengaged_sharerHash, beforetime_DF)
    h_t = engagementHistory(random_userHash, random_valid_article, beforetime_DF)
    h_f = engagementHistory(random_userHash, random_notengaged_article, beforetime_DF)
    
    features.append([s_t, t_t, h_t, 1])
    features.append([s_f, t_f, h_f, 0])
    count = count + 1

In [ ]:
filtered1 = articles_2[articles_2['sharerHash'] != '[]']
filtered2 = filtered1[filtered1['articleTitle'] != 'nan']
filtered3 = filtered2[filtered2['articlePopularity'] != '[]']
filtered4 = filtered3[filtered3['articleComments'] != '[]']

In [ ]:
X = filtered4[['userHash', 'articleTitle', 'sharerHash', 'articlePopularity', 'articleComments', 'articleSeenTime', 'articleClicked']].copy().reset_index()
y = filtered4['articleClicked'].reset_index()

In [ ]:
titles = []
userData = []
comments = []
user_sharer_pops = []
times = []
y = []
p_count = 0
n_count = 0

for i in range(0, len(X)):
    for j in range(0, len(ast.literal_eval(X.iloc[i]["sharerHash"]))):
        if X.iloc[i]["articleClicked"] == True and p_count < 5000:
            p_count = p_count + 1
            titles.append(str(X.iloc[i]["articleTitle"]))
            currUserHash = int(X.iloc[i]["userHash"])
            currUserData = getUserData(X.iloc[i]["userHash"], users)
            userData.append(currUserData)
            comments.append(ast.literal_eval(X.iloc[i]["articleComments"])[j])
            times.append(ast.literal_eval(X.iloc[i]["articleSeenTime"][j]))
            currSharerHash = int(ast.literal_eval(X.iloc[i]["sharerHash"])[j])
            pops_temp = ast.literal_eval(X.iloc[i]["articlePopularity"])
            curr_pop = pops_temp[j]
            try:
                curr_pop= int(curr_pop)
            except:
                if curr_pop.endswith('K'):
                    num_len = len(curr_pop)
                    num = curr_pop[:-1]
                    curr_pop = float(num)*1000
                else:
                    curr_pop = 0
            user_sharer_pops.append([currUserHash, currSharerHash, curr_pop])
            y.append(1)
        elif X.iloc[i]["articleClicked"] == False and n_count < 5000:
            n_count = n_count + 1
            titles.append(str(X.iloc[i]["articleTitle"]))
            currUserHash = int(X.iloc[i]["userHash"])
            currUserData = getUserData(X.iloc[i]["userHash"], users)
            userData.append(currUserData)
            comments.append(ast.literal_eval(X.iloc[i]["articleComments"])[j])
            times.append(ast.literal_eval(X.iloc[i]["articleSeenTime"][j]))
            currSharerHash = int(ast.literal_eval(X.iloc[i]["sharerHash"])[j])
            pops_temp = ast.literal_eval(X.iloc[i]["articlePopularity"])
            curr_pop = pops_temp[j]
            try:
                curr_pop= int(curr_pop)
            except:
                if curr_pop.endswith('K'):
                    num_len = len(curr_pop)
                    num = curr_pop[:-1]
                    curr_pop = float(num)*1000
                else:
                    curr_pop = 0
            user_sharer_pops.append([currUserHash, currSharerHash, curr_pop])
            y.append(0)

In [ ]:
titles_tfidf = TfidfVectorizer(stop_words='english')
titles_output = titles_tfidf.fit_transform(titles)

userdata_tfidf = TfidfVectorizer(stop_words='english')
userdata_output = userdata_tfidf.fit_transform(userData)

comments_tfidf = TfidfVectorizer(stop_words='english')
comments_output = comments_tfidf.fit_transform(comments)

times = [int(i) for dt.timestamp(i) in times]
all_data_temp = hstack([titles_output, userdata_output, comments_output], format='csr')
user_sharer_pops_sp = sparse.csr_matrix(np.asarray(user_sharer_pops))
all_data = hstack([all_data_temp, user_sharer_pops_sp, times], format='csr')

In [ ]:
clf = RandomForestClassifier(max_depth=7, max_features=739, random_state=0)

In [ ]:
y_pred = clf.fit(all_data, y, cross_val=True)

In [ ]:
importances = clf.feature_importances_